In [1]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L12-v2")

C:\Users\fzehr\AppData\Local\Temp\ipykernel_75284\3565532808.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L12-v2")
d:\RAG_e-commerce\RAG_e-commerce\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
query_result = model.embed_query("heelo world")
len(query_result)

AttributeError: 'str' object has no attribute 'embed_query'

In [8]:
from langchain_core.documents import Document
from connect_db import create_connection
from collections import defaultdict


In [1]:
import os
import time
import dotenv
import os

from pinecone import Pinecone, ServerlessSpec
apikey= os.getenv("PINECONE_API_KEY")
pc= Pinecone(api_key = apikey)




d:\RAG_e-commerce\RAG_e-commerce\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
index = pc.Index("ecommerce-2")

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [6]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [9]:
def fetch_data_from_postgres():
    connection = create_connection()
    cursor = connection.cursor()
    query = """
        -- her sipariş kullanıcı ve ürün bilgileir ile birleşir
SELECT
    u.user_id,
    u.user_name AS user_name,
    o.order_id,
    o.product_id,
    o.order_date,
    p.product_name AS product_name,
    p.category AS product_category
FROM
    orders o
JOIN
    users u ON o.user_id = u.user_id
JOIN
    products p ON o.product_id = p.product_id
ORDER BY 
    o.order_date DESC;
        """
    cursor.execute(query)
    data = cursor.fetchall()
    print("veri alınıd")
    connection.close()
    return data



In [10]:
def prepare_docs(data):
    documents = []

    grouped_orders = defaultdict(list)
    for row in data:
        user_id, user_name, order_id, product_id, order_date, product_name, product_category = row

        order_date_trimmed = order_date.replace(microsecond=0)

        key = (user_id, order_date_trimmed)

        if key not in grouped_orders:
            grouped_orders[key] = {
                'user_name': user_name,  # Kullanıcı adı ekleniyor
                'products': []          # Ürünler için boş liste
            }

    # Ürün bilgisi ekleniyor
        grouped_orders[key]['products'].append({
            'product_name': product_name,
            'category': product_category
        })

    print(f"groped orders into {len(grouped_orders)} unique user_date combinations")

    for (user_id, order_date), details in grouped_orders.items():
        user_name = details['user_name']
        products = details['products']
        
        # Ürün bilgilerini birleştir, sadece dict olanları al
        products_text = " and ".join([
            f"{p['product_name']} ({p['category']})"
            for p in products if isinstance(p, dict)
        ])

    
        text = f"User {user_name} ordered {products_text} on {order_date}"
        #print(text)
        # embedding = embeddings.embed_documents(text)
        
        # Metadata hazırla
        metadata = {
            "user_id": user_id,
            "user_name": user_name,
            "order_date": str(order_date),
            "products": [p['product_name'] for p in products if isinstance(p, dict)],
            "categories": [p['category'] for p in products if isinstance(p, dict)]
        }
        # print(metadata)

        doc = Document(
            page_content = text,
            metadata=metadata
        )
        documents.append(doc)

        # unique_id = f"{user_id}_{order_date}"
        # embedding_list.append((unique_id, embedding, metadata))
        
        # embeddings.append((user_id, embedding, metadata))

    return documents

In [1]:

from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from langchain_huggingface import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv
from langchain.chains.question_answering import load_qa_chain
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")
pc = Pinecone(api_key=pinecone_api_key)
index_name= "ecommerce-2"
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=model)
print("pinecone bağlantı başarılı")


d:\RAG_e-commerce\RAG_e-commerce\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
C:\Users\fzehr\AppData\Local\Temp\ipykernel_8992\2711609940.py:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)


pinecone bağlantı başarılı


In [11]:
data = fetch_data_from_postgres()
docs = prepare_docs(data)

connection succes
veri alınıd
groped orders into 140 unique user_date combinations


In [29]:
docs[1]

Document(metadata={'user_id': 346, 'user_name': 'Jacob Fischer', 'order_date': '2025-01-01 14:16:55', 'products': ['Smartwatch (Pink)', 'Plus Size Coat (Green)'], 'categories': ['Electronics', 'Clothing']}, page_content='User Jacob Fischer ordered Smartwatch (Pink) (Electronics) and Plus Size Coat (Green) (Clothing) on 2025-01-01 14:16:55')

In [33]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(docs))]


In [34]:
vector_store.add_documents(documents=docs, ids = uuids)

['f386a5c8-219b-41ee-b120-49a805375946',
 'ccb54243-89d8-4862-a85a-bc3e63974c17',
 'bb9243ac-09a8-482b-a5c2-cafbab09f5ef',
 '4db47027-bf41-485d-bb79-b7819a777e77',
 'd0011354-9351-4298-9625-8f8b8eef44d3',
 'ecf8a6b5-1d60-450a-99e9-28dd586f0a23',
 '49c170e0-e1b5-4f04-a1d9-b17e46cc0106',
 'bdfbc029-9f15-43c5-ac0d-a86ffbfea663',
 '13e6a811-9fad-4eab-9afc-105e6b9088b2',
 'c91bef84-3673-412c-ac57-200263c81d34',
 'be4315b2-3603-4c3d-a239-14056de30f64',
 '18356b7e-e8ee-4796-931e-3d518be6bb4d',
 '27d06b6e-c10f-44e4-8bc5-6e7362dfeea7',
 'fb9ec26b-c798-4468-9741-947730f6ddab',
 'a4196655-ede5-4e7c-a7fa-38cdb2851e4f',
 '29ca8f0a-b7bc-4efe-b5fd-8e21e0128629',
 '99bb6198-be4c-4199-a9d6-c879fe73cfdb',
 'a101f0f3-42fe-430d-b373-266adce1688c',
 '3186bf54-9bc2-4117-9a38-874f4961c201',
 '0318c63d-cb8c-4bb4-9800-811adbc8e6b9',
 'b61121fd-d07f-477a-b542-4a05bd858013',
 'e69f40f6-768e-4a63-8905-7fb927039175',
 'c68b9c1a-f563-4326-a372-94dad2e2f0e0',
 'da094ece-3807-4f40-aa1c-30d770f13f2d',
 '9bc0eb81-80eb-

In [2]:
results = vector_store.similarity_search_with_score(
    "who bus a smartphone",
    k=3,
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.490285] User Cheryl Fuller ordered Smartphone (Yellow) (Electronics) on 2025-01-01 14:11:59 [{'categories': ['Electronics'], 'order_date': '2025-01-01 14:11:59', 'products': ['Smartphone (Yellow)'], 'user_id': 350.0, 'user_name': 'Cheryl Fuller'}]
* [SIM=0.379550] User Marcus Williams ordered Powerbank (Purple) (Electronics) and Smartphone (Red) (Electronics) on 2025-01-01 13:06:39 [{'categories': ['Electronics', 'Electronics'], 'order_date': '2025-01-01 13:06:39', 'products': ['Powerbank (Purple)', 'Smartphone (Red)'], 'user_id': 349.0, 'user_name': 'Marcus Williams'}]
* [SIM=0.373453] User Mr. Joseph Taylor ordered Smartphone (Blue) (Electronics) and Food Processor (Purple) (Home Goods) on 2025-01-01 14:12:56 [{'categories': ['Electronics', 'Home Goods'], 'order_date': '2025-01-01 14:12:56', 'products': ['Smartphone (Blue)', 'Food Processor (Purple)'], 'user_id': 398.0, 'user_name': 'Mr. Joseph Taylor'}]


In [3]:
def retrieve_query(query, k=100):
    matching_docs = vector_store.similarity_search(query, k=k)
    return matching_docs

In [41]:
from langchain.chains.question_answering import load_qa_chain
from langchain_community.chat_models import ChatOpenAI

In [42]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

C:\Users\fzehr\AppData\Local\Temp\ipykernel_51260\1360019337.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)


In [4]:
chain = load_qa_chain(llm,chain_type="stuff")

C:\Users\fzehr\AppData\Local\Temp\ipykernel_8992\3203274189.py:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm,chain_type="stuff")


In [9]:
from langchain.prompts import ChatPromptTemplate

rag_template = """
You are an e-commerce data assistant. Use the context provided to answer the user's query.
Context:
{context}
Question: {question}
Answer:"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)

In [10]:
def retrieve_answer(query):
    doc_search = retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents = doc_search, question = query )
    return response

In [ ]:
pip install --upgrade openai


In [1]:

from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from langchain_huggingface import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv
from langchain.chains.question_answering import load_qa_chain
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")
pc = Pinecone(api_key=pinecone_api_key)
index_name= "ecommerce-2"
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=model)

d:\RAG_e-commerce\RAG_e-commerce\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
C:\Users\fzehr\AppData\Local\Temp\ipykernel_63972\1164457846.py:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)


In [2]:
retriever = vector_store.as_retriever()


In [10]:
print(retriever)

tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'] vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000021E64325E50> search_kwargs={}


In [7]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)

chain = ConversationalRetrievalChain.from_llm(llm, retriever= retriever, memory= memory)


In [9]:
query = 'can you give me tha name of users who buy a Shampoo (Yellow)'
chain.run({'question': query})

'The users who bought Shampoo (Yellow) are Jason Martinez and Kylie Hughes.'

In [34]:
load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

pc = Pinecone(api_key=pinecone_api_key)

In [35]:
pc.create_index(
    name=  "new-ecommerce",
    metric="dotproduct",
    dimension=768,
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1"

    )
)

In [36]:
index = pc.Index("new-ecommerce")

In [2]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from langchain_huggingface import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv
from langchain.chains.question_answering import load_qa_chain
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

In [5]:
#create embedding

pc = Pinecone(api_key=pinecone_api_key)
index_name= "new-ecommerce"
index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [40]:
query = "can you give me tha name of users who buy a Shampoo (Yellow)"
retriever = vector_store.as_retriever(search_kwargs = {"k": 100})

In [41]:
retriever.get_relevant_documents(query)

[Document(id='c166c1f5-4688-470b-8c25-e5a593373d25', metadata={'categories': ['Personal Care'], 'order_date': '2025-01-01 14:10:40', 'products': ['Shampoo (White)'], 'user_id': 341.0, 'user_name': 'Lori Graham'}, page_content='User Lori Graham ordered Shampoo (White) (Personal Care) on 2025-01-01 14:10:40'),
 Document(id='ecf1643d-33cd-4552-87b6-dddc3fa11ef4', metadata={'categories': ['Personal Care', 'Home Goods'], 'order_date': '2025-01-01 13:06:45', 'products': ['Shampoo (Yellow)', 'Knife Set (Red)'], 'user_id': 404.0, 'user_name': 'Jason Martinez'}, page_content='User Jason Martinez ordered Shampoo (Yellow) (Personal Care) and Knife Set (Red) (Home Goods) on 2025-01-01 13:06:45'),
 Document(id='70e5f113-c563-4cf3-b980-d0f428869097', metadata={'categories': ['Jewelry', 'Personal Care'], 'order_date': '2025-01-01 13:06:25', 'products': ['Necklace (Purple)', 'Shampoo (Orange)'], 'user_id': 370.0, 'user_name': 'Bradley Maldonado'}, page_content='User Bradley Maldonado ordered Necklace 

In [35]:
retriever.get_relevant_documents(query)

[Document(id='67b17925-57de-4c8d-8ff3-49462b385a3a', metadata={'categories': ['Electronics'], 'order_date': '2025-01-01 14:11:59', 'products': ['Smartphone (Yellow)'], 'user_id': 350.0, 'user_name': 'Cheryl Fuller'}, page_content='User Cheryl Fuller ordered Smartphone (Yellow) (Electronics) on 2025-01-01 14:11:59'),
 Document(id='9db806a4-035d-479e-9a49-1b1a926365ae', metadata={'categories': ['Electronics', 'Electronics'], 'order_date': '2025-01-01 13:06:39', 'products': ['Powerbank (Purple)', 'Smartphone (Red)'], 'user_id': 349.0, 'user_name': 'Marcus Williams'}, page_content='User Marcus Williams ordered Powerbank (Purple) (Electronics) and Smartphone (Red) (Electronics) on 2025-01-01 13:06:39'),
 Document(id='1bf7c049-87b0-4d5d-be4e-8df5c13b861d', metadata={'categories': ['Electronics', 'Home Goods'], 'order_date': '2025-01-01 14:12:56', 'products': ['Smartphone (Blue)', 'Food Processor (Purple)'], 'user_id': 398.0, 'user_name': 'Mr. Joseph Taylor'}, page_content='User Mr. Joseph Ta

In [17]:
def format_docs(docs):
    return "\n\n".join(str(doc.metadata) for doc in docs)

retrieved_docs = retriever.invoke(query)
print(format_docs(retrieved_docs))

{'categories': ['Electronics'], 'order_date': '2025-01-01 14:11:59', 'products': ['Smartphone (Yellow)'], 'user_id': 350.0, 'user_name': 'Cheryl Fuller'}

{'categories': ['Electronics', 'Electronics'], 'order_date': '2025-01-01 13:06:39', 'products': ['Powerbank (Purple)', 'Smartphone (Red)'], 'user_id': 349.0, 'user_name': 'Marcus Williams'}

{'categories': ['Home Goods', 'Electronics'], 'order_date': '2025-01-01 14:12:02', 'products': ['Table Clock (Orange)', 'Smartphone (Blue)'], 'user_id': 431.0, 'user_name': 'Derek Holt'}


In [20]:
#prompt template

template = """ You are an expert assistant specialized in answering questions related to e-commerce datas. Please analyze the e-commerce data and provide a clear and accurate answer accurately

Guidlines:
1. provide concise and information answer, analyze the e-commerce data
2. if the question is beyond the scope of provided information, say: I dont know.
3. maintain a professional and helpfull tone 

Context : {context}

Questions : {question}


Answer:
"""

In [34]:
prompt = template.format(context = format_docs(retrieved_docs), question = "can you say the names of users that bougth a smartphone")
print(prompt)

 You are an expert assistant specialized in answering questions related to e-commerce datas. Please analyze the e-commerce data and provide a clear and accurate answer accurately

Guidlines:
1. provide concise and information answer
2. if the question is beyond the scope of provided information, say: I dont know.
3. maintain a professional and helpfull tone 

Context : {'categories': ['Electronics'], 'order_date': '2025-01-01 14:11:59', 'products': ['Smartphone (Yellow)'], 'user_id': 350.0, 'user_name': 'Cheryl Fuller'}

{'categories': ['Electronics', 'Electronics'], 'order_date': '2025-01-01 13:06:39', 'products': ['Powerbank (Purple)', 'Smartphone (Red)'], 'user_id': 349.0, 'user_name': 'Marcus Williams'}

{'categories': ['Home Goods', 'Electronics'], 'order_date': '2025-01-01 14:12:02', 'products': ['Table Clock (Orange)', 'Smartphone (Blue)'], 'user_id': 431.0, 'user_name': 'Derek Holt'}

Questions : can you say the names of users that bougth a smartphone


Answer:



In [27]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key= os.getenv("OPENAI_API_KEY"), model="gpt-3.5-turbo", temperature=0)

In [30]:
query = "can you say the names of users that bougth a smartphone"

In [31]:
custom_rag_template = PromptTemplate.from_template(template)

custom_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_template
    | llm
    | StrOutputParser()
)

In [32]:
custom_rag_chain.invoke(query)

'The users who bought a smartphone are:\n1. Cheryl Fuller\n2. Marcus Williams\n3. Mr. Joseph Taylor\n4. Derek Holt\n5. Thomas Cooper\n6. Nancy Morales\n7. William Velasquez\n8. Cynthia Andrade\n9. Jacob Fischer\n10. John Rodriguez\n11. Cheryl Fuller'

In [37]:
import gradio as gr

def llm_response(query, memory= None):
    return custom_rag_chain.invoke(query)

In [39]:
with gr.Blocks() as rag_demo:
    gr.Markdown("# RAG Demo")
    chatbot = gr.Chatbot(height=300)
    textbox = gr.Textbox(placeholder="Enter query here", scale=5)
    clear_btn = gr.Button("Clear")
    undo_btn = gr.Button("Undo")
    retry_btn = gr.Button("Retry")
    submit_btn = gr.Button("Submit")
    
    # Submit işlemi
    def submit(user_query, chat_history):
        response = llm_response(user_query)
        chat_history.append((user_query, response))
        return chat_history, ""
    
    # Undo işlemi
    def undo(chat_history):
        if chat_history:
            chat_history.pop()
        return chat_history
    
    # Clear işlemi
    def clear():
        return [], ""

    # Bağlantılar
    submit_btn.click(submit, inputs=[textbox, chatbot], outputs=[chatbot, textbox])
    undo_btn.click(undo, inputs=chatbot, outputs=chatbot)
    clear_btn.click(clear, outputs=[chatbot, textbox])

rag_demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
custom_rag_chain.invoke()

In [42]:
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder

contextualised_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualised_template = ChatPromptTemplate.from_messages(
    [
        ("system", contextualised_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

In [43]:
#create history
from langchain.chains import create_history_aware_retriever

history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualised_template)

In [44]:
#create_system_prompt

#prompt template

system_prompt_tmeplate = """ You are an expert assistant specialized in answering questions related to e-commerce datas. Please analyze the e-commerce data and provide a clear and accurate answer accurately

Guidlines:
1. Review the data and respond based on the information requested
2. If the question is beyond your knowledge or the information provided, say "I don't know." 
3. Be specific in your responses
4. Use a professional and helpful tone.

Context : {context}

"""


system_prompt_tmeplate = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_tmeplate),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)



In [45]:
# create questions answer chain rag chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

question_answer_chain = create_stuff_documents_chain(llm, system_prompt_tmeplate)
rag_chain = create_retrieval_chain( history_aware_retriever, question_answer_chain) #runnable



In [46]:
#managing chat history
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()


    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [47]:
import pprint

def conversational_chain(query):
    answer = conversational_rag_chain.invoke(
        {"input": query},
        config={
            "configurable": {"session_id": "MySessionId0001"}     
        }
    )
    pprint.pprint(answer)
    return answer

In [52]:
conversational_chain("How many orders has User Jason Martinez placed?")

{'answer': 'User Jason Martinez has placed two orders.',
 'chat_history': [HumanMessage(content='who bougth a jean?', additional_kwargs={}, response_metadata={}),
                  AIMessage(content='The following users bought jeans:\n\n1. Calvin Harrell ordered Straight Jeans (White) (Clothing) on 2025-01-01 14:14:32\n2. Cheryl Fuller ordered Mom Jeans (Brown) (Clothing) and Wool Coat (Yellow) (Clothing) on 2024-12-30 10:24:10\n3. Allison Roberts ordered Jeans (Pink) (Clothing) on 2025-01-01 14:16:41\n4. Diana Nunez ordered Mom Jeans (Brown) (Clothing) and Short Sleeve Shirt (Purple) (Clothing) on 2025-01-01 14:14:17\n5. Kylie Hughes ordered Wool Coat (Pink) (Clothing) on 2025-01-01 14:13:50\n6. Carrie Williams ordered Men Blazer (Pink) (Clothing) on 2025-01-01 14:15:11\n7. Amy Walker ordered Plus Size Coat (Blue) (Clothing) on 2025-01-01 14:12:15\n8. Kathleen Miller ordered V-neck T-shirt (Brown) (Clothing) on 2025-01-01 14:14:55\n9. Jacob Fischer ordered Plus Size Coat (Green) (Clot

{'input': 'How many orders has User Jason Martinez placed?',
 'chat_history': [HumanMessage(content='who bougth a jean?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The following users bought jeans:\n\n1. Calvin Harrell ordered Straight Jeans (White) (Clothing) on 2025-01-01 14:14:32\n2. Cheryl Fuller ordered Mom Jeans (Brown) (Clothing) and Wool Coat (Yellow) (Clothing) on 2024-12-30 10:24:10\n3. Allison Roberts ordered Jeans (Pink) (Clothing) on 2025-01-01 14:16:41\n4. Diana Nunez ordered Mom Jeans (Brown) (Clothing) and Short Sleeve Shirt (Purple) (Clothing) on 2025-01-01 14:14:17\n5. Kylie Hughes ordered Wool Coat (Pink) (Clothing) on 2025-01-01 14:13:50\n6. Carrie Williams ordered Men Blazer (Pink) (Clothing) on 2025-01-01 14:15:11\n7. Amy Walker ordered Plus Size Coat (Blue) (Clothing) on 2025-01-01 14:12:15\n8. Kathleen Miller ordered V-neck T-shirt (Brown) (Clothing) on 2025-01-01 14:14:55\n9. Jacob Fischer ordered Plus Size Coat (Green) (Clothing) on 202

In [53]:
def conversational_chain(query):
    answer = conversational_rag_chain.invoke(
        {"input": query},
        config={
            "configurable": {"session_id": "MySessionId0001"}     
        }
    )["answer"]
    return answer

In [54]:
conversational_chain("How many orders has User Jason Martinez placed?")

'User Jason Martinez has placed two orders.'

In [ ]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import os
from dotenv import load_dotenv
import openai

# Ortam değişkenlerini yükle
load_dotenv()

# Model ve Pinecone kurulumu
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
openai.api_key = os.getenv("OPENAI_API_KEY")
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'e-commerce-npkcsdw.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'e-commerce',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'deletion_protection': 'disabled',
              'dimension': 768,
              'host': 'ecommerce-2-npkcsdw.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'ecommerce-2',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'deletion_protection': 'disabled',
              'dimension': 768,
              'host': 'new-ecommerce-npkcsdw.svc.aped-4627-b74a.pinecone.io',
              'metric': 'dotproduct',
              'name': 'new-ecommerce',
              'spec': {'serverle